In [1]:
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model
from PIL import Image, ImageSequence
import numpy as np

Using TensorFlow backend.
/Users/jamescampbell/anaconda3/envs/env_full/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#  Pre Processing for Training Images and Test Images

def process_tif(filename):
    """
    Opens a tif file, seperates out the individual images.
    Then sets the mean to 0 and std deviation to 1
    """
    tif_image = Image.open(filename)
    images = []
    for i, page in enumerate(ImageSequence.Iterator(tif_image)):
        images.append(np.array(page))
        
    processed_images = np.array(images).astype('float32')
    processed_images = processed_images[..., np.newaxis]
    mean = np.mean(processed_images)  # mean for data centering
    std = np.std(processed_images)  # std for data normalization

    processed_images -= mean
    processed_images /= std
    return images, processed_images


train_original, train_processed = process_tif('train-volume.tif')
test_original, test_processed = process_tif('test-volume.tif')

In [3]:
#  Pre Processing for Training Labels

labels = Image.open('train-labels.tif')
train_labels = []

for i, page in enumerate(ImageSequence.Iterator(labels)):
    train_labels.append(np.array(page))
    
train_mask = np.array(train_labels).astype('float32')
train_mask = train_mask[..., np.newaxis]

train_mask /= 255.  # scale masks to [0, 1]
    
train_mask[train_mask > 0.5] = 1
train_mask[train_mask <= 0.5] = 0

In [4]:
img_rows = 512
img_cols = 512

In [ ]:
def get_unet():

    inputs = Input(shape=(img_rows, img_cols, 1))

    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu',padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu',padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu',padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu',padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu',padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu',padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2DTranspose(512, 2, strides=2, padding='same', kernel_initializer='he_normal')(drop5)
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2DTranspose(256, 2, strides=2, padding='same', kernel_initializer='he_normal')(conv6)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2DTranspose(128, 2, strides=2, padding='same', kernel_initializer='he_normal')(conv7)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2DTranspose(64, 2, strides=2, padding='same', kernel_initializer='he_normal')(conv8)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, output=conv10)

    model.compile(optimizer=Adam(lr=1e-4, ), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
#  Train the model
#  You don't actually need to do this, you can just load a pre-trained one below
model = get_unet()
model.fit(train_processed, train_mask, batch_size=1, epochs=10, verbose=1)

In [ ]:
#  Save the model so we don't have to keep training
model.save('saved-unet-model.h5')

In [5]:
#  Load a pretrained model
from keras.models import load_model
model = load_model('saved-unet-model.h5')

In [6]:
#  Create Predictions for the training images
results = model.predict(train_processed, batch_size=1, verbose=1)
results = np.squeeze(results)
results[results > 0.5] = 1
results[results <= 0.5] = 0
results *= 255

30/30 [==============================] - 477s 16s/step


In [10]:
#  Compare the original training image, the labelled image and the predicted label
y = 7
Image.fromarray(results[y]).show()
Image.fromarray(train_original[y]).show()
Image.fromarray(train_labels[y]).show()

In [9]:
#  Create predictions for the test images
testing = model.predict(test_processed, batch_size=1, verbose=1)
testing = np.squeeze(testing)
testing[testing > 0.5] = 1
testing[testing <= 0.5] = 0
testing *= 255

30/30 [==============================] - 388s 13s/step


In [13]:
#  Compare the original test image with the predicted label
x = 10
Image.fromarray(testing[x]).show()
Image.fromarray(test_original[x]).show()